# Préparer un voyage avec Kayak

## Récupération des données météo
Pour récupérer les données météo, on utilisera l'API d'Open Weather (https://openweathermap.org/api).

Or celle-ci fonctionne avec des coordonnées GPS, il faut donc récupérer pour chaque ville sa latitude et sa longitude.

On utilisera alors l'API https://nominatim.org/release-docs/develop/api/Search/ qui permet de récupérer des coordonnés en fonction d'une recherche par addresse.

In [8]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px

try:
    df = pd.read_csv("data/df.csv", sep=';', index_col=0)
except:
    df = pd.read_json("data/cities.json")
    df = df.rename(columns={0 : "City"})
    df["Latitude"] = np.nan
    df["Longitude"] = np.nan
    df["Prain"] = np.nan
    df["Vrain"] = np.nan
    df["Temp"] = np.nan


In [2]:
def get_coordinates(city):
    """Return tuple(lat, long)"""

    display(f"get_coordinates {city}")
    r = requests.get("https://nominatim.openstreetmap.org/search", params= {"city": city, "country": "FRANCE", "format": "jsonv2"})
    if (r.status_code == 200):
        json = r.json()[0]
        return (json["lat"], json["lon"])
    else:
        raise ValueError("API did not return 200")

In [19]:
for index, row in df.iterrows():
    if (pd.isna(row["Latitude"]) == False and pd.isna(row["Longitude"]) == False):
        continue

    try:
        coordinates = get_coordinates(row["City"])
        display(coordinates)
        df.loc[df['City'] == row["City"], ['Latitude']] = coordinates[0]
        df.loc[df['City'] == row["City"], ['Longitude']] = coordinates[1]
    except ValueError as e:
       display(f"{row['City']} : {e}")


df

,City,Latitude,Longitude,Prain,Vrain,Temp
0,Mont Saint Michel,48.635954,-1.511460,NaN,NaN,NaN
1,St Malo,48.649518,-2.026041,NaN,NaN,NaN
2,Bayeux,49.276462,-0.702474,NaN,NaN,NaN
3,Le Havre,49.493898,0.107973,NaN,NaN,NaN
4,Rouen,49.440459,1.093966,NaN,NaN,NaN
5,Paris,48.858890,2.320041,NaN,NaN,NaN
6,Amiens,49.894171,2.295695,NaN,NaN,NaN
7,Lille,50.636565,3.063528,NaN,NaN,NaN
8,Strasbourg,48.584614,7.750713,NaN,NaN,NaN
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,NaN,NaN,NaN


Maintenant que l'on a recuéperer les coordonnées GPS de chaque ville, on peut récupérer les infomations météorologique de ces villes dans les 5 prochains jours.

In [8]:
weather_api_key = "WEATHER_API_KEY"
weather_domain = 'https://api.openweathermap.org'

In [33]:
def get_weather(city, lat, long):
    """Return tuple(rain probabilty, rain volume in mm, day temperature)"""

    print(f"get_weather {city}")
    r = requests.get(f"{weather_domain}/data/2.5/forecast", params= {"lat": lat, "lon": long, "units": "metric", "appid": weather_api_key, "mode": "json"})
    display(r.content)
    if (r.status_code == 200):
        json = r.json()["list"][len(r.json()["list"]) - 1] # On récupère la dernière prevision (celle de j+5)
        pop = json["pop"]
        rain = json["rain"]["3h"] if "rain" in json else 0
        temp = json["main"]["temp"]
        return (pop, rain, temp)
    else:
        raise ValueError("API did not return 200")

In [ ]:
for index, row in df.iterrows():
    if (pd.isna(row["Prain"]) == False and pd.isna(row["Vrain"]) == False and pd.isna(row["Temp"]) == False):
        continue

    try:
        weather = get_weather(row["City"], row["Latitude"], row["Longitude"])
        display(weather)
        df.loc[df['City'] == row["City"], ['Prain']] = weather[0]
        df.loc[df['City'] == row["City"], ['Vrain']] = weather[1]
        df.loc[df['City'] == row["City"], ['Temp']] = weather[2]
    except ValueError as e:
       display(f"{row['City']} : {e}")


df

In [35]:
# save to csv to not recall API when the kernel restart
df.to_csv("data/df.csv", sep=';')

Alors qu'elles est la meilleur destination ?

In [5]:
df["size_bubble"] = 1 - df["Prain"]
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", color="Temp", 
                        zoom=3.5, size="size_bubble", size_max=15, mapbox_style="carto-positron", 
                        color_continuous_scale="Bluered", color_continuous_midpoint=22)
fig.show()


In [ ]:
import json

f = open('./data/cities.json') 
cities = json.load(f)

for city in cities:
    city_without_white_space = city.replace(' ', '_')
    !python ./src/scrapy_booking.py $city_without_white_space

![Alt text](image.png)

In [ ]:
import boto3
from sqlalchemy import create_engine
from sqlalchemy import types 

engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

bucket = "jedha-project-kayak"
s3_client = boto3.client(
    "s3",
    aws_access_key_id="aws_access_key_id",
    aws_secret_access_key="aws_secret_access_key",
)

list_files = s3_client.list_objects_v2(
    Bucket=bucket,
    Prefix="booking/"
)

for item in list_files['Contents']:
    key = item['Key']

    response = s3_client.get_object(Bucket=bucket, Key=key)

    hotels_df = pd.read_json(response.get("Body"))

    hotels_df["price"] = hotels_df["price"].str.replace("\n", '')
    hotels_df["price"] = hotels_df["price"].str.replace("€", '')
    hotels_df["price"] = hotels_df["price"].str.replace(" ", '')
    hotels_df["price"] = hotels_df["price"].str.strip()

    hotels_df["score"]  = hotels_df["score"].str.replace(",",".")
    pd.to_numeric(hotels_df["score"])
    hotels_df["score"] = hotels_df['score'].fillna(hotels_df['score'].median())
    hotels_df.to_sql(
        "hotels", 
        engine, 
        if_exists="append", 
        index=False,
        dtype={
            'score' : types.FLOAT(0),
            'latitude' : types.FLOAT(0), 
            'longitude' : types.FLOAT(0), 
            'price' : types.FLOAT(0)
        }
    )

print("end")



![Alt text](image-1.png)
![Alt text](image-2.png)
![Alt text](image-3.png)